# 모듈 설치

In [1]:
! pip install cssselect

In [1]:
import lxml.html
import requests
from selenium.webdriver import Chrome

In [10]:
import pandas as pd

# 주소 가져오기

In [2]:
url = 'https://www.epeople.go.kr/nep/pttn/gnrlPttn/pttnSmlrCaseList.npaid?pageIndex={}'

In [10]:
title_list = []
#browser = Chrome()
#browser.get(url)
for page in range(1,10001) :
    #browser.get(url.format(page))
    res = requests.get(url.format(page))
    root = lxml.html.fromstring(res.text)
    titles = root.cssselect('td.left a')
    for i in range(len(titles)) :
        a = titles[i].text_content() 
        title_list.append(a)

In [13]:
final = pd.DataFrame(title_list, columns=['title'])
final.to_csv('홍.csv',encoding='utf-8')

# 파일 합치기

In [3]:
a = pd.read_csv('홍.csv',encoding='utf-8')
b = pd.read_csv('유라.csv',encoding='utf-8')
c = pd.read_csv('준수.csv',encoding='utf-8')
d = pd.read_csv('혜은.csv',encoding='utf-8')
e = pd.read_csv('세홍.csv',encoding='utf-8')

In [22]:
len(b), len(c),len(d),len(e)

(99990, 99990, 99990, 31409)

In [4]:
total = pd.concat([a,b,c,d,e])

In [5]:
total.iloc[:,[1]]

,title
0,높이 31m를 넘는 8개 층의 각층별 바닥면적이 400㎡인 경우 비상용승강기 설치대...
1,비상용승강기 승강장을 통한 피난계단의 출입구 가능 여부
2,승강기가 서지 않는 층의 승강장에 승강기문을 설치하지 않아도 되는지
3,소규모 들녘경영체 지원
4,유기농 생태마을 운영 활성화 지원
...,...
31404,예방수칙 변경
31405,기부체납 도로
31406,화성시 목동 동탄2택지 개발지구 A49블록 동탄2신도시호반베르디움 504동 동명칭변경의건
31407,마을 길의 파인 부분으로 인하여 통행이 상당히 불편한 상황입니다.


In [6]:
final = total.drop_duplicates(['title'], keep='first')

In [7]:
len(final)

133768

In [8]:
final.to_csv('중복제거_real.csv',encoding='utf-8')

# 검색 키워드 넣어서 크롤링(feat.셀레늄)

In [2]:
from selenium import webdriver

In [3]:
url = 'https://www.epeople.go.kr/nep/pttn/gnrlPttn/pttnSmlrCaseList.npaid'

In [4]:
browser = Chrome()
browser.get(url)

In [5]:
element = browser.find_element_by_name('searchWord')
element.send_keys("주차") #주차/정차 바꿔주기
element.submit()

In [6]:
#주차 > 266 > 준수오빠
#정차 > 2063 > 홍(1~400) / 호진(401~800) / 유라(801~1200) / 혜은언니(1201~1600) / 세홍(1601~2063)

In [7]:
title_list = []

for i in range(1,266+1) :
    titles = browser.find_elements_by_css_selector('td.left a')
    
    for j in range(len(titles)) : 
        a = titles[j].text
        title_list.append(a)
        
    next_page = browser.find_elements_by_css_selector('span.nep_p_next > a')
    next_page[0].click()    

In [17]:
# 10*할당된 페이지수 정도가 나오면 ㅇㅋ

len(title_list)
import pandas as pd

In [13]:
final = pd.DataFrame(title_list, columns=['title'])

# 이름 > 자기 이름으로 바꾸세요 ^^
final.to_csv('준수.csv',encoding='utf-8')